# 📘 Data Merging & Cleaning – Yousra Descriptive Stats Notebook

This notebook loads the original project datasets (sales, weather, kiwo event),  
cleans and standardizes the date format, merges them using a full outer join,  
and performs descriptive statistics, missing value inspection, and prepares the  
data for further feature engineering and modeling.

The goal is to:
1. Combine **all available data** on matching dates  
2. Perform a clear descriptive exploration  
3. Detect and handle missing values  
4. Produce a clean dataset ready for analysis  


In [ ]:
import pandas as pd
import numpy as np

# Load data from your project folder
df_kiwo = pd.read_csv("data/kiwo.csv")
df_weather = pd.read_csv("data/wetter.csv")
df_umsatz = pd.read_csv("data/umsatzdaten_gekuerzt.csv")

df_kiwo.head(), df_weather.head(), df_umsatz.head()
#print number of rows for each dataframe
print(f"Kiwo Data Rows: {len(df_kiwo)}")
print(f"Weather Data Rows: {len(df_weather)}")
print(f"Umsatz Data Rows: {len(df_umsatz)}")

In [ ]:
# Convert 'Datum' to proper datetime format and drop invalid rows
for df in (df_kiwo, df_weather, df_umsatz):
    df["Datum"] = pd.to_datetime(df["Datum"], errors="coerce").dt.normalize()
    df.dropna(subset=["Datum"], inplace=True)

    # Drop duplicate date columns if exist
    if "date" in df.columns:
        df.drop(columns=["date"], inplace=True)

df_kiwo.info(), df_weather.info(), df_umsatz.info()


In [ ]:
# Adding suffixes helps avoid confusion during merging

df_kiwo = df_kiwo.add_suffix("_kiwo")
df_kiwo.rename(columns={"Datum_kiwo": "Datum"}, inplace=True)

df_weather = df_weather.add_suffix("_weather")
df_weather.rename(columns={"Datum_weather": "Datum"}, inplace=True)


df_umsatz = df_umsatz.add_suffix("_umsatz")
df_umsatz.rename(columns={"Datum_umsatz": "Datum"}, inplace=True)

df_umsatz["umsatz_rolling7"] = (
    df_umsatz["Umsatz_umsatz"].rolling(window=7, min_periods=1).mean()
)


In [ ]:
merged_df = (
    df_kiwo
    .merge(df_weather, on="Datum", how="outer")
    .merge(df_umsatz, on="Datum", how="outer")
)

merged_df.shape
merged_df.head()


In [ ]:
# Numeric summary
merged_df.describe()

# For all columns (including categorical)
merged_df.describe(include="all")


In [ ]:
missing_count = merged_df.isna().sum()
missing_percent = (merged_df.isna().sum() / len(merged_df)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent
})

missing_df


### ✔️ Missing Value Strategy

**Umsatz (Sales):**
- Missing values mean the day has **no sales** or is **outside the bakery event period**.
- These rows should be **removed**, not imputed.

**Weather Variables:**
- Weather data may be missing because not all dates have weather records.
- Fill missing values using interpolation (numerical)  
  and assign -1 for Wettercode to indicate “unknown weather”.

**Kiwo Event Flag:**
- If missing → fill with 0 (day outside event).


In [ ]:
clean_df = merged_df.copy()

# Remove rows where sales are missing
clean_df = clean_df.dropna(subset=["Umsatz_umsatz"])

# Fill missing Wettercode with category -1
if "Wettercode_weather" in clean_df.columns:
    clean_df["Wettercode_weather"] = clean_df["Wettercode_weather"].fillna(-1)

# Interpolate numerical weather values
for col in clean_df.columns:
    if ("_weather" in col) and (clean_df[col].dtype in ["float64", "int64"]):
        clean_df[col] = clean_df[col].interpolate()

clean_df.isna().sum()


In [ ]:
clean_df["KielerWoche_kiwo"] = clean_df["KielerWoche_kiwo"].fillna(0)


In [ ]:
clean_df.isna().sum()


In [ ]:
#drop wettercode_weather column
df_no_weather_code = clean_df.drop(columns=["Wettercode_weather"])
df_no_weather_code.sample(5)


## Adding extra columns

In [ ]:
df_no_weather_code['Datum'] = pd.to_datetime(df_no_weather_code['Datum'], errors='coerce').dt.normalize()
n_invalid = df_no_weather_code['Datum'].isna().sum()
if n_invalid > 0:
    print(f"Warning: {n_invalid} rows have invalid 'Datum' and will have NaT in day columns. Sample:")
    display(df_no_weather_code[df_no_weather_code['Datum'].isna()].head())

# Create integer and name columns for day of week
df_no_weather_code['day_of_week'] = df_no_weather_code['Datum'].dt.weekday  # Monday=0 .. Sunday=6
df_no_weather_code['day'] = df_no_weather_code['Datum'].dt.day_name()

# Optional: make 'day' categorical ordered Monday..Sunday
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_no_weather_code['day'] = pd.Categorical(df_no_weather_code['day'], categories=ordered_days, ordered=True)

print("\nCounts per weekday:")
print(df_no_weather_code['day'].value_counts().sort_index())
df_no_weather_code.sample(5)

### Adding more weather data from meteo archive api

In [ ]:
import requests
from datetime import datetime, date, timedelta
import time
from typing import Union, Iterable, Dict, List
import pandas as pd
import numpy as np

# Simple in-memory cache for API responses
try:
    _OPEN_METEO_CACHE
except NameError:
    _OPEN_METEO_CACHE: Dict[str, Dict[str, float]] = {}

def _iso_date(d: Union[str, date, datetime]) -> str:
    if isinstance(d, str):
        return datetime.fromisoformat(d).date().isoformat()
    if isinstance(d, datetime):
        return d.date().isoformat()
    return d.isoformat()

def fetch_open_meteo_daily_range(
    start_date: Union[str, date, datetime],
    end_date: Union[str, date, datetime],
    latitude: float = 54.3233,
    longitude: float = 10.1228,
    timezone: str = "Europe/Berlin",
    daily_vars: List[str] = None,
    max_retries: int = 3,
    timeout: float = 15.0,
    use_cache: bool = True
) -> pd.DataFrame:
    """
    Fetch daily historical variables from Open-Meteo archive API for a given inclusive date range.
    By default uses Kiel coordinates (lat=54.3233, lon=10.1228).
    daily_vars example: ['precipitation_hours', 'sunshine_duration', 'rain_sum']
    Returns a pandas.DataFrame indexed by date (datetime.date) with columns named after daily_vars.
    """
    if daily_vars is None:
        daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum']

    start_iso = _iso_date(start_date)
    end_iso = _iso_date(end_date)
    start_dt = datetime.fromisoformat(start_iso).date()
    end_dt = datetime.fromisoformat(end_iso).date()
    if end_dt < start_dt:
        raise ValueError("end_date must be >= start_date")

    # Build cache key based on parameters and requested range
    key = f"{latitude:.6f}_{longitude:.6f}_{start_iso}_{end_iso}_{','.join(daily_vars)}_{timezone}"
    if use_cache and key in _OPEN_METEO_CACHE:
        df = _OPEN_METEO_CACHE[key].copy()
        return df

    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_iso,
        "end_date": end_iso,
        "daily": ",".join(daily_vars),
        "timezone": timezone
    }

    last_exc = None
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(base_url, params=params, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            daily = data.get("daily", {})
            times = daily.get("time", []) or []
            if not times:
                raise ValueError("No 'time' in API response daily block.")
            results = {"date": pd.to_datetime(times).date}
            for v in daily_vars:
                vals = daily.get(v, None)
                if vals is None:
                    # If a requested variable is missing, fill with NaN
                    results[v] = [np.nan] * len(times)
                else:
                    # Convert to numeric (floats), keep NaN if parsing fails
                    results[v] = [float(x) if x is not None else np.nan for x in vals]
            df = pd.DataFrame(results, index=pd.to_datetime(times).date)
            df.index.name = "date"
            df = df[[v for v in daily_vars]]  # ensure column order
            if use_cache:
                _OPEN_METEO_CACHE[key] = df.copy()
            return df
        except Exception as e:
            last_exc = e
            if attempt < max_retries:
                time.sleep(0.5 * attempt)
                continue
            raise RuntimeError(f"Failed fetching Open-Meteo archive: {e}") from e


In [ ]:
min_date = df_no_weather_code['Datum'].min()
max_date = df_no_weather_code['Datum'].max()
##min_date = "2013-07-01"
##max_date = "2014-07-30"
print (f"Fetching weather data from {min_date} to {max_date}...")
lat_kiel, lon_kiel = 54.3233, 10.1228
daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum','temperature_2m_mean']
df_extra_weather = fetch_open_meteo_daily_range(min_date, max_date, latitude=lat_kiel, longitude=lon_kiel, daily_vars=daily_vars)
# print columns of df_extended_weather


In [ ]:
df_extra_weather['sunshine_hours'] = df_extra_weather['sunshine_duration'] / 3600.0
#df_extended_weather.head()

# print row count for df_extended_weather and df_no_weather_code
print(f"Extended Weather Data Rows: {len(df_extra_weather)}")
print(f"Sales Data Rows: {len(df_no_weather_code)}")


### Extended Data Frames
Its with extra weather data from open meteo api: df_merged_extended_weather
with rolling 7 day average as well for umsatz

In [44]:
df_extra_weather.index = pd.to_datetime(df_extra_weather.index)
df_merged_extended_weather = df_no_weather_code.merge(
    df_extra_weather,
    left_on='Datum',
    right_index=True,
    how='left'
)

print(f"Merged DataFrame Rows: {len(df_merged_extended_weather)}")
print(f"Merged DataFrame Shape: {df_merged_extended_weather.shape}")

df_merged_extended_weather = df_merged_extended_weather.dropna(subset=["id_umsatz"])
df_merged_extended_weather["id_umsatz"] = df_merged_extended_weather["id_umsatz"].astype("int64")
df_merged_extended_weather = df_merged_extended_weather.dropna(subset=["KielerWoche_kiwo"])
df_merged_extended_weather["KielerWoche_kiwo"] = df_merged_extended_weather["KielerWoche_kiwo"].astype("int64")

df_merged_extended_weather.head()

Merged DataFrame Rows: 9334
Merged DataFrame Shape: (9334, 16)


,Datum,KielerWoche_kiwo,Bewoelkung_weather,Temperatur_weather,Windgeschwindigkeit_weather,id_umsatz,Warengruppe_umsatz,Umsatz_umsatz,umsatz_rolling7,day_of_week,day,precipitation_hours,sunshine_duration,rain_sum,temperature_2m_mean,sunshine_hours
394,2013-07-01,0,6.0,17.8375,15.0,1307011,1.0,148.828353,148.828353,0,Monday,7.0,31025.67,1.0,15.6,8.618242
395,2013-07-01,0,6.0,17.8375,15.0,1307012,2.0,535.856285,221.786925,0,Monday,7.0,31025.67,1.0,15.6,8.618242
396,2013-07-01,0,6.0,17.8375,15.0,1307013,3.0,201.198426,592.534505,0,Monday,7.0,31025.67,1.0,15.6,8.618242
397,2013-07-01,0,6.0,17.8375,15.0,1307014,4.0,65.890169,311.369828,0,Monday,7.0,31025.67,1.0,15.6,8.618242
398,2013-07-01,0,6.0,17.8375,15.0,1307015,5.0,317.475875,109.314600,0,Monday,7.0,31025.67,1.0,15.6,8.618242


# Testing Regression

In [ ]:
df_merged_extended_weather.tail()


In [ ]:
import scipy.stats as stats
import seaborn as sns
# Testing Regression
reg_cols=['Umsatz_umsatz','umsatz_rolling7','KielerWoche_kiwo','Bewoelkung_weather', 'rain_sum', 'sunshine_hours','Temperatur_weather','Windgeschwindigkeit_weather','precipitation_hours','day_of_week']
sns.pairplot(df_merged_extended_weather[reg_cols].dropna())
df_merged_extended_weather[reg_cols].corr()

In [26]:
import statsmodels.formula.api as smf

# Example regression formula (adjust as needed)
formula = "Umsatz_umsatz ~  Temperatur_weather  + C(Warengruppe_umsatz)"

# Drop rows with missing values in the regression columns
regression_df = df_merged_extended_weather.dropna(subset=[
    "Umsatz_umsatz", "Bewoelkung_weather", "rain_sum", "sunshine_hours",
    "Temperatur_weather", "Windgeschwindigkeit_weather", "precipitation_hours", "day_of_week","Warengruppe_umsatz"
])

# Fit the model
model = smf.ols(formula, data=regression_df).fit()

# Show summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Umsatz_umsatz   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     3379.
Date:                Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                        21:20:44   Log-Likelihood:                -54278.
No. Observations:                9334   AIC:                         1.086e+05
Df Residuals:                    9327   BIC:                         1.086e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       72.7318      2.381     30.542      0.000      68.064      77.400
C(Warengruppe_umsatz)[T.2.0]   280.3570      2.691    104.170      0.000     275.081     285.633
C(Warengruppe_umsatz)[T.3.0]    41.2017      2.691     15.309      0.000      35.926      46.477
C(Warengruppe_umsatz)[T.4.0]   -34.0141      2.711    -12.545      0.000     -39.329     -28.699
C(Warengruppe_umsatz)[T.5.0]   154.6595      2.691     57.466      0.000     149.384     159.935
C(Warengruppe_umsatz)[T.6.0]   -32.9995      5.157     -6.399      0.000     -43.109     -22.890
Temperatur_weather               4.0474      0.117     34.516      0.000       3.818       4.277
==============================================================================
Omnibus:                     8941.438   Durbin-Watson:                   1.511
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1520347.829
Skew:                           4.162   Prob(JB):                         0.00
Kurtosis:                      64.967   Cond. No.                         96.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Merging test data from test csv


In [28]:
# Read test.csv into testdf
df_test = pd.read_csv("data/test.csv")

# Quick checks
print("Shape:", df_test.shape)
display(df_test.head())
display(df_test.tail())

Shape: (1830, 3)


,id,Datum,Warengruppe
0,1808011,2018-08-01,1
1,1808021,2018-08-02,1
2,1808031,2018-08-03,1
3,1808041,2018-08-04,1
4,1808051,2018-08-05,1


,id,Datum,Warengruppe
1825,1812226,2018-12-22,6
1826,1812236,2018-12-23,6
1827,1812246,2018-12-24,6
1828,1812276,2018-12-27,6
1829,1812286,2018-12-28,6


In [49]:
## add weather Temperatur_weather from df_weather
df_test['Datum'] = pd.to_datetime(df_test['Datum'], errors='coerce').dt.normalize()
df_weather['Datum'] = pd.to_datetime(df_weather['Datum'], errors='coerce').dt.normalize()


# Left-join temperature from df_weather onto the test DataFrame (keep all test rows)
df_test_weather = df_test.merge(
    df_weather[['Datum', 'Temperatur_weather']],
    on='Datum',
    how='left'
)

# Quick sanity checks
print("Shape:", df_test_weather.shape)
print("Missing Temperatur_weather:", df_test_weather['Temperatur_weather'].isna().sum())
display(df_test_weather.head())
display(df_test_weather.tail())

Shape: (1830, 4)
Missing Temperatur_weather: 65


,id,Datum,Warengruppe,Temperatur_weather
0,1808011,2018-08-01,1,23.7625
1,1808021,2018-08-02,1,26.1875
2,1808031,2018-08-03,1,27.6625
3,1808041,2018-08-04,1,25.1375
4,1808051,2018-08-05,1,21.3000


,id,Datum,Warengruppe,Temperatur_weather
1825,1812226,2018-12-22,6,4.3000
1826,1812236,2018-12-23,6,6.4500
1827,1812246,2018-12-24,6,2.5000
1828,1812276,2018-12-27,6,7.1250
1829,1812286,2018-12-28,6,7.3125


## Predicting with df_test_weather

In [62]:
#predicting with df_test_weather
# rename df_test_weather.Warengruppe to Warengruppe_umsatz for consistency
df_test_weather = df_test_weather.rename(columns={"Warengruppe": "Warengruppe_umsatz"})

# Required columns for this model
req_cols = ['Temperatur_weather', 'Warengruppe_umsatz']

# Sanity checks
missing = [c for c in req_cols if c not in df_test_weather.columns]
if missing:
    raise ValueError(f"Missing columns for prediction: {missing}")

# Ensure numeric temperature
df_test_weather['Temperatur_weather'] = pd.to_numeric(df_test_weather['Temperatur_weather'], errors='coerce')





# Drop rows missing predictor values
#df_predicted = df_test_weather.dropna(subset=req_cols).copy()
df_predicted = df_test_weather.copy()



# Predict in next cell
#df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())





(1830, 4)


,id,Datum,Warengruppe_umsatz,Temperatur_weather
0,1808011,2018-08-01,1,23.7625
1,1808021,2018-08-02,1,26.1875
2,1808031,2018-08-03,1,27.6625
3,1808041,2018-08-04,1,25.1375
4,1808051,2018-08-05,1,21.3000


,id,Datum,Warengruppe_umsatz,Temperatur_weather
1825,1812226,2018-12-22,6,4.3000
1826,1812236,2018-12-23,6,6.4500
1827,1812246,2018-12-24,6,2.5000
1828,1812276,2018-12-27,6,7.1250
1829,1812286,2018-12-28,6,7.3125


In [63]:

#predicting with df_test_weather
df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())

(1830, 5)


,id,Datum,Warengruppe_umsatz,Temperatur_weather,predicted_Umsatz_umsatz
0,1808011,2018-08-01,1,23.7625,168.907770
1,1808021,2018-08-02,1,26.1875,178.722675
2,1808031,2018-08-03,1,27.6625,184.692565
3,1808041,2018-08-04,1,25.1375,174.472922
4,1808051,2018-08-05,1,21.3000,158.941089


,id,Datum,Warengruppe_umsatz,Temperatur_weather,predicted_Umsatz_umsatz
1825,1812226,2018-12-22,6,4.3000,57.136086
1826,1812236,2018-12-23,6,6.4500,65.837960
1827,1812246,2018-12-24,6,2.5000,49.850796
1828,1812276,2018-12-27,6,7.1250,68.569944
1829,1812286,2018-12-28,6,7.3125,69.328828


## generating data submission csv from df_predicted
Will write a csv file in data named predicted.csv with these columns id,umsatz

In [64]:
# Prepare output frame and clean missing rows
df_out_predicted = df_predicted[["id", "predicted_Umsatz_umsatz"]].copy()
#rename columns accoding to kaggle submission requirements
df_out_predicted.rename(columns={ "predicted_Umsatz_umsatz": 'umsatz'}, inplace=True)

#print columns name df_out_predicted
print(df_out_predicted.columns)
print(df_out_predicted.shape)

# write to csv
df_out_predicted.to_csv("data/predicted.csv", index=False)
print("Wrote data/predicted.csv with", len(df_out_predicted), "rows.")




Index(['id', 'umsatz'], dtype='object')
(1830, 2)
Wrote data/predicted.csv with 1830 rows.
